<a href="https://colab.research.google.com/github/satyam123457/Dsg-project-final-/blob/main/satyam_sinha_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WE WILL BE FIRST STARTING WITH FIRST TWO AIMS OF OUR PROJECT** 
which are :-

 


1.Recommending the 10 most popular songs 

2.Recommending top 10 user songs based on user id 

Recommenders.py is also an algorithm used here,code for which is  provided in the dsg end evaluation doc file 

In [14]:
import pandas as pd
import numpy as np
import Recommenders as Recommenders
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf


*Importing the first 3 algorithms -- cosine similiarity algorithm, count vectoriser and TSNE*

## Displaying the 2 datasets 

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
 
import warnings
warnings.filterwarnings('ignore')

#next line 
song_df_1 = pd.read_csv('triplets_file.csv')
song_df_1.head()



,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0


In [16]:
song_df_2 = pd.read_csv('song_data1.csv')
song_df_2.head()


,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003.0
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995.0
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006.0
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003.0
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0.0


Combine both datasets based on a common column of **song_id**

In [17]:
# combine both data
song_df = pd.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on='song_id', how='left')
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,NaN,NaN,NaN,NaN
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,NaN,NaN,NaN,NaN
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,NaN,NaN,NaN,NaN


 creating new feature combining **title** and **artist_name**




In [18]:

song_df['song'] = song_df['title']+' - '+song_df['artist_name']
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,NaN,NaN,NaN,NaN,NaN
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,NaN,NaN,NaN,NaN,NaN
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,NaN,NaN,NaN,NaN,NaN


# taking top 10k samples for quick results

In [19]:
song_df = song_df.head(10000)


# cummulative sum of listen count of the songs

In [20]:

song_grouped = song_df.groupby(['song']).agg({'listen_count':'count'}).reset_index()
song_grouped.head()

,song,listen_count
0,#40 - DAVE MATTHEWS BAND,1
1,(Anaesthesia) Pulling Teath - Metallica,1
2,14 Years - Guns N' Roses,1
3,143 - Musiq,2
4,42 - Coldplay,2


#Taking sum of listen count and finding the percentage listen count of every user id 

Sorting the percentages

In [21]:
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage'] = (song_grouped['listen_count'] / grouped_sum ) * 100
song_grouped.sort_values(['listen_count', 'song'], ascending=[0,1])

,song,listen_count,percentage
887,Undo - Björk,32,1.651187
205,Dog Days Are Over (Radio Edit) - Florence + Th...,28,1.444788
477,Lucky (Album Version) - Jason Mraz & Colbie Ca...,20,1.031992
545,Nothin' On You [feat. Bruno Mars] (Album Versi...,16,0.825593
173,Cosmic Love - Florence + The Machine,15,0.773994
...,...,...,...
980,Your Woman - White Town,1,0.051600
981,Zero - The Pain Machinery,1,0.051600
984,high fives - Four Tet,1,0.051600
985,paranoid android - Christopher O'Riley,1,0.051600


# display the top 10 popular songs

In [22]:
pr = Recommenders.popularity_recommender_py()
pr.create(song_df, 'user_id', 'song')

pr.recommend(song_df['user_id'][5])

,user_id,song,score,Rank
887,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,32,1.0
205,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,28,2.0
477,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Lucky (Album Version) - Jason Mraz & Colbie Ca...,20,3.0
545,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Nothin' On You [feat. Bruno Mars] (Album Versi...,16,4.0
173,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Cosmic Love - Florence + The Machine,15,5.0
307,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Halo - Beyoncé,15,6.0
588,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Party In The U.S.A. - Miley Cyrus,14,7.0
310,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Harder Better Faster Stronger - Daft Punk,13,8.0
814,b80344d063b5ccb3212f76538f3d9e43d87dca9e,The Only Exception (Album Version) - Paramore,12,9.0
28,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Ain't No Rest For The Wicked (Original Version...,11,10.0


aim 2 of the project (based on user id ) started 

In [23]:
ir = Recommenders.item_similarity_recommender_py()
ir.create(song_df, 'user_id', 'song')
user_items = ir.get_user_items(song_df['user_id'][5])
# display user songs history
for user_item in user_items:
  print(user_item) 

nan
Entre Dos Aguas - Paco De Lucia
Stronger - Kanye West
Heaven's gonna burn your eyes - Thievery Corporation feat. Emiliana Torrini
Let It Be Sung - Jack Johnson / Matt Costa / Zach Gill / Dan Lebowitz / Steve Adams
I'll Be Missing You (Featuring Faith Evans & 112)(Album Version) - Puff Daddy
Behind The Sea [Live In Chicago] - Panic At The Disco
Oh No - Andrew Bird
Love Song For No One - John Mayer
The Christmas Song  (LP Version) - King Curtis
Drive - Incubus


**# give song recommendation for that user based on user id**


In [24]:
ir.recommend(song_df['user_id'][5])

No. of unique songs for the user: 11
no. of unique songs in the training set: 990
Non zero values in cooccurence_matrix :293


,user_id,song,score,rank
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Plástico - Ruben Blades,0.045455,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Drunken Lullabies - Flogging Molly,0.034091,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Crazy For You No Kisetsu (Album Version) - Bas...,0.030303,3
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,DONTTRUSTME (Explicit Album Version) - 3OH!3,0.030303,4
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,The Warrior's Code - Dropkick Murphys,0.030303,5
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Love Me - Justin Bieber,0.030303,6
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dark Matter - Andrew Bird,0.030303,7
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Not Falling Apart - Maroon 5,0.030303,8
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Ddiamondd - Battles,0.030303,9
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Once In A Lifetime (Remastered LP Version ) - ...,0.030303,10




```
# This is formatted as code
```

#3rd aim  initialise - recommend songs based on songs enter 


In [25]:
spoti = pd.read_csv("genres_v21.csv", encoding='utf-8', quotechar='"')
spoti.head(3)



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,Unnamed: 13,Unnamed: 14,Unnamed: 15,duration_ms,time_signature,genre,song_name,Unnamed: 0,title
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,...,2Vc6NJ9PW9gD9q343XFRKx,NaN,NaN,NaN,124539,4,Dark Trap,Mercury: Retrograde,NaN,NaN
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,...,7pgJBLVz5VmnL7uGHmRj6p,NaN,NaN,NaN,224427,4,Dark Trap,Pathology,NaN,NaN
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,...,0vSWgAlfpye0WCGeNmuNhy,NaN,NaN,NaN,98821,4,Dark Trap,Symbiote,NaN,NaN


In [26]:
spoti.shape

(42305, 22)

In [27]:
song_name = spoti["song_name"]
print(song_name.shape)
print(song_name.isnull().values.any())

(42305,)
True


In [28]:
song_name = song_name.values.reshape(-1,1)
song_name.shape

(42305, 1)

USING SIMPLE IMPUTER TO CLEAR OUT NULL VALUES

In [29]:
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr = imr.fit(song_name)
imputed_data = imr.transform(song_name)
song_name = pd.DataFrame(imputed_data)
song_name = song_name.rename(columns={0:"Song-Names"})
song_name.head(3)

,Song-Names
0,Mercury: Retrograde
1,Pathology
2,Symbiote


In [30]:
core = spoti[["genre","mode","duration_ms"]]
print(core.dtypes)
print(core.head())

genre          object
mode            int64
duration_ms     int64
dtype: object
       genre  mode  duration_ms
0  Dark Trap     1       124539
1  Dark Trap     1       224427
2  Dark Trap     1        98821
3  Dark Trap     1       123661
4  Dark Trap     1       123298


In [31]:
core.dtypes


genre          object
mode            int64
duration_ms     int64
dtype: object

In [32]:
core["genre"].value_counts()

Underground Rap    5875
Dark Trap          4578
Hiphop             3028
trance             2999
trap               2987
techhouse          2975
dnb                2966
psytrance          2961
techno             2956
hardstyle          2936
RnB                2099
Trap Metal         1956
Rap                1848
Emo                1680
Pop                 461
Name: genre, dtype: int64

In [33]:
core = core.replace({"genre":{"Underground Rap":0, "Dark Trap":1, "Hiphop":2, "trance":3, "trap":4, "techhouse":5, "dnb":6, "psytrance": 7, "techno":8, "hardstyle":9, "RnB":10, "Trap Metal":11, "Rap":12, "Emo":13, "Pop":14}})

USING NON NEGATIVE MATRIX FACTORIZATION

In [34]:
from sklearn.decomposition import NMF
nmf = NMF(n_components = 6)


In [35]:
nmf_features = nmf.fit_transform(core)
from sklearn.preprocessing import normalize
norm_features = normalize(nmf_features)
current_music = norm_features[23,:]
similarities = norm_features.dot(current_music)
similarities

array([0.97654121, 0.14562393, 0.99638084, ..., 0.26140112, 0.16906763,
       0.67629962])

In [36]:
df = pd.DataFrame(norm_features)
x = df.join(song_name)
df = pd.pivot_table(x, x[[0,1,2,3,4,5]],["Song-Names"])#for indexing song_name to our df
def current_music(value):
    print("Top 5 recommendations for given music are:")
    value = df.loc[value]
    similarities = df.dot(value)
    print(format(similarities.nlargest()))

Recommending the final results based on song entered

In [37]:
current_music("Missed Calls - Remix")

Top 5 recommendations for given music are:
Song-Names
Missed Calls - Remix           1.000000
Sig 9                          0.999482
Dead Angels                    0.999272
Money Up (feat. Toro y Moi)    0.998984
Gaahlish                       0.998942
dtype: float64


In [38]:
current_music("SEARCH & DESTRXY.")

Top 5 recommendations for given music are:
Song-Names
SEARCH & DESTRXY.                              1.000000
Angel With a Shotgun                           0.999999
All the Things She Said (Barnacle Boi Flip)    0.999998
FXRTUNE.                                       0.999997
Red                                            0.999987
dtype: float64


In [39]:
current_music("Animals")

Top 5 recommendations for given music are:
Song-Names
Bug & Snake                        0.888993
Da Grind (Featuring Apocalypse)    0.888826
Painting Shadows                   0.888718
Black Bandana                      0.888673
Sick Ritual                        0.888618
dtype: float64
